## Dependencies

In [503]:
!pip install clean-text
!pip install inflect

In [504]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [505]:
datatype = 'train' 

path = (
    'PIZZA_train.json' if datatype == 'train' else
    'PIZZA_dev.json' if datatype == 'dev' else
    'PIZZA_test.json' if datatype == 'test' else
    'unknown.json'  # Optional fallback
)

lines = (
    2457303 if datatype == 'train' else
    348 if datatype == 'dev' else
    500 if datatype == 'test' else
    0  
)

print(f"Path: {path}")
print(f"Lines: {lines}")


Path: PIZZA_train.json
Lines: 2457303


## Parsing JSON File

In [506]:

#! load data given file path and number of line to load
def load_data(data, file_path,num_lines):
    # data = []
    with open(file_path, 'r') as file: 
        for _ in range(num_lines):
            data.append(json.loads(file.readline()))
            # i+=1
    #         if i%100000==0 or (i>2450000):
    #             print(i) 
    # print(f"Loaded {num_lines} lines")
    # return data
# def load_random_data(file_path, num_lines, last_lines):
#     data = []
    
#     with open(file_path, 'r') as file:
#         file.seek(0, 2)  # Move to the end of the file to get its size
#         file_size = file.tell()
        
#         # Read the last `last_lines` lines
#         file.seek(max(file_size - 1024 * last_lines, 0))  # Optional: Adjust the seek window size for efficiency
#         lines = file.readlines()
#         last_lines_data = [json.loads(line.strip()) for line in lines[-last_lines:] if line.strip()]
#         data.extend(last_lines_data)

#         # Read `num_lines` random lines from the rest of the file (excluding the last `last_lines` lines)
#         remaining_lines = file_size - sum(len(line) for line in lines[-last_lines:])
#         for _ in range(num_lines):
#             while True:
#                 # Pick a random position outside the last `last_lines` range
#                 random_pos = random.randint(0, remaining_lines - 1)
#                 file.seek(random_pos)
                
#                 # Read to the end of the current line to avoid partial lines
#                 file.readline()
#                 line = file.readline()  # Read the next line (complete line)
#                 if line:  # Ensure it's not an empty line
#                     # Check if it's not from the last `last_lines` lines
#                     # if any(line.strip() == last_line for last_line in last_lines_data):
#                     #     continue  # Skip if it's from the last lines
#                     try:
#                         data.append(json.loads(line.strip()))
#                         break  # Exit the loop for this line
#                     except json.JSONDecodeError:
#                         continue  # Skip if it's not valid JSON
#     return data


In [507]:
data=[]
load_data(data, path, lines)
print(f"Loaded {len(data)} lines")
print(data[:5])

Loaded 2457303 lines
[{'train.SRC': 'can i have a large bbq pulled pork', 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (TOPPING BBQ_PULLED_PORK ) ) )', 'train.TOP': '(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )', 'train.TOP-DECOUPLED': '(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )'}, {'train.SRC': 'large pie with green pepper and with extra peperonni', 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (TOPPING GREEN_PEPPERS ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING PEPPERONI ) ) ) )', 'train.TOP': '(ORDER (PIZZAORDER (SIZE large ) pie with (TOPPING green pepper ) and with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING peperonni ) ) ) )', 'train.TOP-DECOUPLED': '(ORDER (PIZZAORDER (SIZE large ) (TOPPING green pepper ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING peperonni ) ) ) )'}, {'train.SRC': "i'd like a large vegetarian pizza", 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (

In [508]:
#! split json data
def getextensions(datatype):
    if datatype=='train':
        return 'train.SRC','train.EXR','train.TOP'
    elif datatype=='dev':
        return 'dev.SRC','dev.EXR','dev.TOP'
    else:
        return 'test.SRC','test.EXR','test.TOP'
def get_training_data(data,datatype='train'):
    values=getextensions(datatype)
    training_data = []
    training_exr=[]
    training_top=[]
    # training_top_dec=[]
    for item in data:
        training_data.append(item[values[0]])  
        training_exr.append(item[values[1]])  
        training_top.append(item[values[2]])
        # if datatype=='train':
        #     training_top_dec.append(item[values[3]])  
    return training_data,training_exr,training_top
training_data,training_exr,training_top=get_training_data(data,datatype)   
print(training_data[:2])
print(training_exr[:2])
print(training_top[:2])


['can i have a large bbq pulled pork', 'large pie with green pepper and with extra peperonni']
['(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (TOPPING BBQ_PULLED_PORK ) ) )', '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (TOPPING GREEN_PEPPERS ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING PEPPERONI ) ) ) )']
['(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )', '(ORDER (PIZZAORDER (SIZE large ) pie with (TOPPING green pepper ) and with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING peperonni ) ) ) )']


In [509]:
# Create an inflect engine
p = inflect.engine()

def replace_numbers_with_words(text):
    # Regular expression to find numbers in the text
    number_pattern = r'\b\d+\b'
    
    # Function to convert a number match to its word representation
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')

    
    # Substitute numbers with their word representation
    text = re.sub(number_pattern, number_to_words, text)
    
    # Replace the standalone 'a' with 'one' only when it reprsents a
    text = re.sub(
        r'\b[aA]\b(?! ((lot)|(little)|(few)|(number)|(couple)|(number)|(bit)|(number)|(bit)|(load)|(stack)|(bunch)|(group)|(set)|(series)|(variety)|(range)|(number)|(amount)|(sum)|(total)))',
        'one',
        text
    )
    text=re.sub(r'\ban\b', 'one', text)
    return text

In [510]:
text = "an onion, a lot of effort, a little joy, a book, a bunch of cheese, a dog."
print(replace_numbers_with_words(text))


one onion, a lot of effort, a little joy, one book, a bunch of cheese, one dog.


In [511]:
#! a function to handle negations
def handle_negations(text):
    #! negations are based on training set
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

In [512]:
#!# lemmatize words with all possible pos tags
def lem_word(word):
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [513]:
lemmatizer = WordNetLemmatizer()
#! stopwords list (adding not_)
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
# stopwords.update(not_stopwords)
# stopwords.discard('a')
# stopwords.discard('an')
# stopwords.discard('not')
# stopwords.discard('no')
# stopwords.discard('can')
# stopwords.discard('not_a')
# stopwords.discard('not_an')
# stopwords.discard('not_can')
# stopwords.discard('not_no')
stopwords=set()
# stopwords.add('and')
# stopwords.add('also')

In [514]:
#! tokenize training data and remove stop words
def preprocess_training_data(training_data, stopwords):
    # training_data = [handle_negations(order) for order in training_data]
    training_data=[replace_numbers_with_words(order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    training_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in training_data]
    training_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in training_data]
    # print(training_data)
    #! remove d letter most probably garbage
    training_data = [re.sub(r'\bd\s+', '', order) for order in training_data]
    #! remove "can" at the beginning of the sentence
    # training_data=[re.sub(r'^can\s+', '', order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    # training_data = [[lemmatizer.lemmatize(word) for word in order] for order in training_data]
    training_data = [[lem_word(word) for word in order] for order in training_data]
    return training_data
training_data = preprocess_training_data(training_data, stopwords)
print(training_data[:5])

[['can', 'i', 'have', 'one', 'large', 'bbq', 'pull', 'pork'], ['large', 'pie', 'with', 'green', 'pepper', 'and', 'with', 'extra', 'peperonni'], ['i', 'like', 'one', 'large', 'vegetarian', 'pizza'], ['party', 'size', 'stuff', 'crust', 'pie', 'with', 'american', 'cheese', 'and', 'with', 'mushroom'], ['can', 'i', 'have', 'one', 'personal', 'size', 'artichoke']]


In [515]:
# def get_vocab_freq(data):
#     vocab_freq = {}
#     for d in data:
#         for w in d:
#             if w in vocab_freq:
#                 vocab_freq[w] += 1
#             else:
#                 vocab_freq[w] = 1
#     return vocab_freq
# vocab_freq= get_vocab_freq(data)
# print(type(vocab_freq))
# def replace_with_unk(data, vocab_freq, threshold):
#     unk = '<UNK>'
#     data_unk = []
#     for d in data:
#         d_unk = [w if vocab_freq[w] >= threshold else unk for w in d]
#         data_unk.append(d_unk)
#     return data_unk
# # training_data = replace_with_unk(training_data, vocab_freq, 50)
# # print(training_data[:5])

In [516]:
# vocab= set()
# for d in training_data:
#     vocab.update(d)
# vocab.add('<PAD>')
# vocab.add('<UNK>')
# print(len(vocab))
# print(vocab)
# word2idx = {w: i for i, w in enumerate(vocab)}

In [517]:
# #! save vocab and word2idx
# with open('vocab.txt', 'w') as file:
#     for w in vocab:
#         file.write(f'{w}\n')
# with open('word2idx.json', 'w') as file:
#     json.dump(word2idx, file)

In [518]:
#! a utility function for extra parentheses ) removal 
#! handles COMPLEX_TOPPING, NOT,... parenthesis cases
def remove_unmatched_parentheses(input_string):
    result = list(input_string)  # Convert to list for mutability
    last_bracket_index=-1
    for i, char in enumerate(result):
        if char == ')' and i+2 < len(result):
            result[i] = ''  
            last_bracket_index=i
        elif char == '(':
            if last_bracket_index!=-1:
                result[last_bracket_index] = ')'
                last_bracket_index=-1
        elif char == ')' and i+2 >= len(result):
            result[i] = ''
            result[last_bracket_index] = ')'
    return ''.join(result)

In [519]:
#! get PIZZAORDER, DRINKORDER, NONE Labels 0=>PIZZAORDER, 1=>DRINKORDER, 2=>NONE

def get_order_category_labels(training_top, training_data, stopwords):
    # print(training_data)
    order_category_labels = []
    for i, item in enumerate(training_top):
        order_category_labels.append([2] * len(training_data[i]))
        #! remove any extra labelling
        item = re.sub(r'\)[^()]*\(', ') (', item)
        # print(item)
        unwanted_keywords = r"\b(ORDER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY|NOT|NUMBER|DRINKTYPE|CONTAINERTYPE|VOLUME)\b"
        cleaned_string = re.sub('\('+unwanted_keywords, "", item)
        cleaned_string = [word for word in cleaned_string.split() if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_string)
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        order_regex = r"\((?:PIZZAORDER|DRINKORDER).*?\)"
        extracted_orders = re.findall(order_regex, cleaned_string)
        k = 0
        for order in extracted_orders:
            order=replace_numbers_with_words(order)
            order = re.sub(r"[\(\)]", "", order)
            order=word_tokenize(order) #! fix id and don't bugs
            order=clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            order=re.sub(r'\bd\s+', '', order) #! for d removal
            tokens = word_tokenize(order)
            # tokens = [lemmatizer.lemmatize(word) for word in tokens]
            tokens = [lem_word(word) for word in tokens]
            # print(tokens)
            j = 0
            to_index_train=training_data[i][k:]
            if 'pizzaorder' in tokens:
                tokens.remove('pizzaorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 0
                        j += 1
                    k += 1
            elif 'drinkorder' in tokens:
                tokens.remove('drinkorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 1
                        j += 1
                    k += 1
    return order_category_labels

order_category_labels = get_order_category_labels(training_top, training_data, stopwords)
print(order_category_labels[:5])
    

 

[[2, 2, 2, 0, 0, 0, 0, 0], [0, 2, 2, 0, 0, 2, 2, 0, 0], [2, 2, 0, 0, 0, 0], [0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0], [2, 2, 2, 0, 0, 0, 0]]


In [520]:
#! save processed training dataset
path = (
    'training_data_processed.txt' if datatype == 'train' else
    'dev_data_processed.txt' if datatype == 'dev' else
    'test_data_processed.txt' if datatype == 'test' else
    'unknown_data_processed.txt'  # Optional fallback
)

with open(path, 'a') as f: #### dev_data_processed.txt
    for item in training_data:
        f.write("%s\n" % item)

In [521]:
path = (
    'train_order_category_labels.txt' if datatype == 'train' else
    'dev_order_category_labels.txt' if datatype == 'dev' else
    'test_order_category_labels.txt' if datatype == 'test' else
    'unknown_order_category_labels.txt'  # Optional fallback
)
for labels in order_category_labels:
    with open(path, 'a') as f: # dev
        f.write("%s\n" % labels)